In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split

from tensorflow.keras.optimizers.legacy import Adam

from functools import partial 

from sklearn.ensemble import GradientBoostingRegressor
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, Dense
from tensorflow.keras.models import Sequential

In [34]:
# file_dir = '/Users/yashwanthkaruparthi/Documents/Acads/sem7/design project/execution/data/solar_weather copy 2.csv'
file_dir = '/Users/yashwanthkaruparthi/Documents/Acads/sem7/design project/execution/data/uae_data_2022.csv'
time_step = 24

# units = 250
# learning_rate = 0.001
# dropout_rate = 0.2
# batch_size = 64

num_feats = 5

In [35]:
import tensorflow.keras.backend as K

# def create_dataset(dataset, time_step):
#     print(f'dataset shape {dataset.shape}')
#     dataX, dataY = [], []
#     for i in range(len(dataset) - time_step):
#         a = dataset[i:(i + time_step), :]  # Features: GHI and Energy delta
#         dataX.append(a)
#         dataY.append(dataset[i + time_step, 0])  # Target: Energy delta
#     return np.array(dataX), np.array(dataY)

def create_dataset (X, y, time_steps):
    Xs, ys = [], []
    
    for i in range(len(X)-time_steps):
        v = X[i:i+time_steps, :]
        Xs.append(v)
        ys.append(y[i+time_steps])
        
    return np.array(Xs), np.array(ys)

def r2_metric(y_true, y_pred):
    """
    R-squared metric for regression tasks.
    """
    ss_res = K.sum(K.square(y_true - y_pred))  # Residual sum of squares
    ss_tot = K.sum(K.square(y_true - K.mean(y_true)))  # Total sum of squares
    return 1 - (ss_res / (ss_tot + K.epsilon()))  # R² formula

### unwanted code

In [36]:
# data = pd.read_csv(file_dir, header=0, infer_datetime_format=True, parse_dates=['Time'], index_col=['Time'])

# # data = data[(data.index.month.isin([6, 7, 8])) & (data.index.year == 2021)]

# data = data[(data.index.year == 2021)]

# dataset = data[['Energy delta[Wh]', 'GHI', 'temp', 'pressure', 'humidity']]
# # dataset = data[['Energy delta[Wh]', 'GHI']]

# X = dataset.iloc[:, 1:].values  # Features
# y = dataset.iloc[:, 0].values   # Target

In [37]:
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train_size = int(len(X) * 0.8)
# test_size = len(X) - train_size
# X_train, X_test = X[:train_size], X[train_size:]
# y_train, y_test = y[:train_size], y[train_size:]

# gbdt = GradientBoostingRegressor(n_estimators=100, learning_rate=learning_rate, max_depth=5)

# gbdt.fit(X_train, y_train.ravel())

# # Step 2: Get predictions for all models on both training and test sets
# gbdt_output_train = gbdt.predict(X_train)
# gbdt_output_test = gbdt.predict(X_test)

# scaler = MinMaxScaler()
# gbdt_output_train_scaled = scaler.fit_transform(gbdt_output_train.reshape(-1, 1))
# gbdt_output_test_scaled = scaler.transform(gbdt_output_test.reshape(-1, 1))

# X_train_bilstm_gbdt, y_train_bilstm_gbdt = create_dataset(gbdt_output_train_scaled, time_step)
# X_test_bilstm_gbdt, y_test_bilstm_gbdt = create_dataset(gbdt_output_test_scaled, time_step)

# X_train_bilstm_gbdt = X_train_bilstm_gbdt.reshape(X_train_bilstm_gbdt.shape[0], time_step, 1)
# X_test_bilstm_gbdt = X_test_bilstm_gbdt.reshape(X_test_bilstm_gbdt.shape[0], time_step, 1)

# model = Sequential([
#   Bidirectional(LSTM(int(units), activation='relu',return_sequences=True, input_shape=(time_step, num_feats))),
#   Dropout(dropout_rate),
#   Bidirectional(LSTM(int(units), activation='relu',return_sequences=False)),
#   Dropout(dropout_rate),
#   Dense(1)
# ])

# model.compile(optimizer='adam', loss='mean_squared_error', metrics=[r2_metric])


# # Train the model
# history = model.fit(X_train_bilstm_gbdt, y_train_bilstm_gbdt, epochs=10, batch_size=int(batch_size), validation_data=(X_test_bilstm_gbdt, y_test_bilstm_gbdt), verbose=1)

### woa bilstm

In [38]:

data = pd.read_csv(file_dir, header=0, infer_datetime_format=True, parse_dates=['Datetime'], index_col=['Datetime'])

year = 2022

data = data[(data.index.year == year)]

# dataset = data[['Temperature', 'DNI', 'DHI', 'GHI', 'Pressure', 'Wind Speed']]
dataset = data[['Temperature', 'GHI', 'Pressure', 'Wind Speed']]

train_size = int(len(dataset)*0.8)
train_dataset, test_dataset = dataset.iloc[:train_size], dataset.iloc[train_size:]

# Split train data to X and y
X_trainy = train_dataset.drop('GHI', axis = 1)
y_trainy = train_dataset.loc[:,['GHI']]

# Split test data to X and y
X_testy = test_dataset.drop('GHI', axis = 1)
y_testy = test_dataset.loc[:,['GHI']]

# Step 1: Train GBDT, XGB, and CatBoost on the training data
gbdt = GradientBoostingRegressor(n_estimators=100, learning_rate=0.001, max_depth=5)

gbdt.fit(X_trainy, y_trainy)

# Step 2: Get predictions for all models on both training and test sets
gbdt_output_train = gbdt.predict(X_trainy)
gbdt_output_test = gbdt.predict(X_testy)

input_scaler = MinMaxScaler()
output_scaler = MinMaxScaler()

input_scaler.fit(gbdt_output_train.reshape(-1, 1))
# output_scaler.fit(gbdt_output_train.reshape(-1, 1))

train_x_norm = input_scaler.transform(gbdt_output_train.reshape(-1, 1))
test_x_norm = input_scaler.transform(gbdt_output_test.reshape(-1, 1))

output_scaler.fit(y_trainy)

train_y_norm = output_scaler.transform(y_trainy)
test_y_norm = output_scaler.transform(y_testy)

/var/folders/n5/mlsmdgns0kqfybw1r9qm4n5w0000gp/T/ipykernel_7488/897731756.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data = pd.read_csv(file_dir, header=0, infer_datetime_format=True, parse_dates=['Datetime'], index_col=['Datetime'])
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


In [39]:


# Define the objective function
def evaluate_bilstm(hyperparameters):
    # Unpack hyperparameters
    units, learning_rate, dropout_rate, batch_size = hyperparameters
    # hyperparameters = [hyperparameters] if not isinstance(hyperparameters, (list, tuple)) else hyperparameters

    # units, learning_rate, dropout_rate, batch_size = (hyperparameters + [None] * 4)[:4]

    
    # params = {"units": units, "learning_rate": learning_rate, "dropout_rate": dropout_rate, "batch_size": batch_size}
    # missing_params = [name for name, value in params.items() if value is None]

    # if missing_params: 
    #     raise ValueError(f'Missing required params: {",".join(missing_params)}')


    print(units, learning_rate, dropout_rate, batch_size)


    train_size = int(len(dataset)*0.8)
    train_dataset, test_dataset = dataset.iloc[:train_size], dataset.iloc[train_size:]

    # Split train data to X and y
    X_trainy = train_dataset.drop('GHI', axis = 1)
    y_trainy = train_dataset.loc[:,['GHI']]

    # Split test data to X and y
    X_testy = test_dataset.drop('GHI', axis = 1)
    y_testy = test_dataset.loc[:,['GHI']]

    gbdt = GradientBoostingRegressor(n_estimators=100, learning_rate=learning_rate, max_depth=5)

    gbdt.fit(X_trainy, y_trainy)

    # Step 2: Get predictions for all models on both training and test sets
    gbdt_output_train = gbdt.predict(X_trainy)
    gbdt_output_test = gbdt.predict(X_testy)

    input_scaler = MinMaxScaler()
    output_scaler = MinMaxScaler()

    input_scaler.fit(gbdt_output_train.reshape(-1, 1))

    train_x_norm = input_scaler.transform(gbdt_output_train.reshape(-1, 1))
    test_x_norm = input_scaler.transform(gbdt_output_test.reshape(-1, 1))

    output_scaler.fit(y_trainy)

    train_y_norm = output_scaler.transform(y_trainy)
    test_y_norm = output_scaler.transform(y_testy)

    # gbdt_output_train_scaled = scaler.fit_transform(gbdt_output_train.reshape(-1, 1))
    # gbdt_output_test_scaled = scaler.transform(gbdt_output_test.reshape(-1, 1))

    TIME_STEPS = 24
    X_train, y_train = create_dataset(train_x_norm, train_y_norm, TIME_STEPS)
    X_test, y_test = create_dataset(test_x_norm, test_y_norm, TIME_STEPS)

    # X_train_bilstm_gbdt, y_train_bilstm_gbdt = create_dataset(gbdt_output_train_scaled, time_step)
    # X_test_bilstm_gbdt, y_test_bilstm_gbdt = create_dataset(gbdt_output_test_scaled, time_step)

    # X_train_bilstm_gbdt = X_train_bilstm_gbdt.reshape(X_train_bilstm_gbdt.shape[0], time_step, 1)
    # X_test_bilstm_gbdt = X_test_bilstm_gbdt.reshape(X_test_bilstm_gbdt.shape[0], time_step, 1)

    model = Sequential([
      Bidirectional(LSTM(int(units), activation='relu',return_sequences=True, input_shape=(time_step, num_feats))),
      Dropout(dropout_rate),
      Bidirectional(LSTM(int(units), activation='relu',return_sequences=False)),
      Dropout(dropout_rate),
      Dense(1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=[r2_metric])

    history = model.fit(X_train, y_train, epochs=10, batch_size=batch_size, validation_data=(X_test, y_test), verbose=1)
    # Train the model
    # history = model.fit(X_train_bilstm_gbdt, y_train_bilstm_gbdt, epochs=10, batch_size=int(batch_size), validation_data=(X_test_bilstm_gbdt, y_test_bilstm_gbdt), verbose=1)

    # Return validation loss as fitness
    return history.history['val_loss'][-1]


In [40]:
lb = [200, 1e-3, 0.1, 10]  # Lower bounds: [units, learning rate, dropout rate, batch size]
ub = [600, 1e-2, 0.4, 300]  # Upper bounds: [units, learning rate, dropout rate, batch size]


In [41]:
from functools import partial
import numpy as np



class WOA:
    def __init__(self, obj_func, n_whale, spiral_constant, n_iter,
                lb, ub):
        self.obj_func = obj_func
        self.n_whale = n_whale
        self.spiral_constant = spiral_constant
        self.n_iter = n_iter
        self.lb = lb
        self.ub = ub
        self.whale = {}
        self.prey = {}

    def init_whale(self):
        tmp = [np.random.uniform(self.lb, self.ub, size=(len(self.lb),))
               for i in range(self.n_whale)]
        self.whale['position'] = np.array(tmp)
        self.whale['fitness'] = np.array([self.obj_func(pos) for pos in self.whale['position']])

    def init_prey(self):
        tmp = [np.random.uniform(self.lb, self.ub, size=(len(self.lb),))]
        self.prey['position'] = np.array(tmp)
        self.prey['fitness'] = self.obj_func(self.prey['position'])

    def update_prey(self):
        if self.whale['fitness'].min() < self.prey['fitness'][0]:
            self.prey['position'][0] = self.whale['position'][self.whale['fitness'].argmin()]
            self.prey['fitness'][0] = self.whale['fitness'].min()

    def search(self, idx, A, C):
        random_whale = self.whale['position'][np.random.randint(low=0, high=self.n_whale,
                                                                size=len(idx[0]))]
        d = np.abs(C[..., np.newaxis] * random_whale - self.whale['position'][idx])
        self.whale['position'][idx] = np.clip(random_whale - A[..., np.newaxis] * d, self.lb, self.ub)

    def encircle(self, idx, A, C):
        d = np.abs(C[..., np.newaxis] * self.prey['position'] - self.whale['position'][idx])
        self.whale['position'][idx] = np.clip(self.prey['position'][0] - A[..., np.newaxis] * d, self.lb, self.ub)

    def bubble_net(self, idx):
        d_prime = np.abs(self.prey['position'] - self.whale['position'][idx])
        l = np.random.uniform(-1, 1, size=len(idx[0]))
        self.whale["position"][idx] = np.clip(
            d_prime * np.exp(self.spiral_constant * l)[..., np.newaxis] * np.cos(2 * np.pi * l)[..., np.newaxis]
            + self.prey["position"],
            self.lb,
            self.ub,
        )

    def optimize(self, a):

        p = np.random.random(self.n_whale)
        r1 = np.random.random(self.n_whale)
        r2 = np.random.random(self.n_whale)
        A = 2 * a * r1 - a
        C = 2 * r2
        search_idx = np.where((p < 0.5) & (abs(A) > 1))
        encircle_idx = np.where((p < 0.5) & (abs(A) <= 1))
        bubbleNet_idx = np.where(p >= 0.5)
        self.search(search_idx, A[search_idx], C[search_idx])
        self.encircle(encircle_idx, A[encircle_idx], C[encircle_idx])
        self.bubble_net(bubbleNet_idx)
        self.whale['fitness'] = self.obj_func(self.whale['position'])

    def run(self):
        self.init_whale()
        self.init_prey()
        f_values = [self.prey['fitness'][0]]
        for n in range(self.n_iter):
            #print("Iteration = ", n, " f(x) = ", self.prey['fitness'][0])
            a = 2 - n * (2 / self.n_iter)
            self.optimize(a)
            self.update_prey()
            f_values.append(self.prey['fitness'][0])
        optimal_x = self.prey['position'].squeeze()
        return f_values, optimal_x

In [42]:
# Partial function to adapt objective function for WOA
# obj_func = partial(lambda params: [evaluate_bilstm(param) for param in params])

# obj_func = lambda params: evaluate_bilstm(params)

obj_func = partial(lambda params: evaluate_bilstm(params))

# Instantiate WOA
woa = WOA(obj_func=obj_func, n_whale=10, spiral_constant=1, n_iter=20, lb=lb, ub=ub)

# Run WOA optimization
f_values, optimal_hyperparameters = woa.run()

print("Optimal Hyperparameters:", optimal_hyperparameters)


216.2375368171455 0.00492850079135253 0.39435588633525953 124.05873887048331


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


TypeError: Expected int32 passed to parameter 'size' of op 'Slice', got [496.23495548193324] of type 'list' instead. Error: Expected int32, but got 496.23495548193324 of type 'float64'.